# Import

In [1]:
# imports
import json
import math
import bisect
import numpy as np
import pandas as pd
import mediapipe as mp
import ipywidgets as widgets
import tensorflow as tf
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from mediapipe.framework.formats import landmark_pb2
from fastai.vision.all import show_image
from ipywidgets import interact, interactive, fixed, interact_manual
from collections import Counter
from collections import defaultdict

# functions
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

KeyboardInterrupt: 

# Data

In [ ]:
# path to a single data file
file = '33432165'
path_dataset = '/Volumes/My Passport for Mac/asl-fingerspelling/supplemental_landmarks/' + file + '.parquet'
path_dataset_labels = '/Volumes/My Passport for Mac/asl-fingerspelling/supplemental_metadata.csv'
dataset = pd.read_parquet(path_dataset)
dataset_labels = pd.read_csv(path_dataset_labels)

## Video Selection

In [ ]:
# Video ID
VideoID = dataset.index.unique() # 1000 total videos

# Select a video
s = 4
seq_id = VideoID[s]  

# Select video data
data = dataset.loc[ seq_id ]
data_labels = dataset_labels.loc[ dataset_labels['sequence_id'] == seq_id ]
phrase = data_labels.phrase.iloc[0]

# head
data_labels.head()

# Data Labels
### Functions

In [ ]:
# List of characters present in the sentence
def character_list(sentence):
    ans = []
    words = sentence.split()
    for word in words:
        prev = ''
        for i in word:
            if i in prev or not prev: 
                prev += i
            else:
                ans += [prev]
                prev = i       
        ans += [ prev ]
    return ans

In [ ]:
# Columns info
Signers = dataset_labels.participant_id.unique() # All signers
Sentences = dataset_labels.phrase.unique() # All sentences 

# Characters counts
Sentences_Characters = [character_list(i) for i in Sentences] # Convert each sentence into list of characters
Character_Counts = Counter( sum(Sentences_Characters, []) ) # Count characters across all sentences

# Convert to a sorted list
Characters = sorted(list(Character_Counts))

# Encoding and decoding characters as integers
Encode, Decode = {}, {}
for i,x in enumerate(Characters):
    Encode[x] = i
    Decode[i] = x

## Exploritory Data Analysis

In total, there are approximately 50,000 videos in the data set which are done by 72 distinct participants. For each video, the participant will spell out the letters of a sentence that is provided to them. The sentences are often reused, so there are multiple videos of different participant spelling out the same sentence. 

Here is a quick summary of the data:

* Total Videos: 52,958
* Total Participants: 72
* Total Distinct Sentences: 508
* Total Distinct Characters: 40

Below is a plot for number of videos each signer performed. 

In [ ]:
# Counts
signers_count = dataset_labels['participant_id'].value_counts() # Count of videos per signer
sentences_count = dataset_labels['phrase'].value_counts() # Count of videos per sentence

# plotting number of videos per signer
ax = signers_count.plot(kind='barh', figsize=(5,20))

### Character Plot

In [ ]:
Characters

In [ ]:
sns.set(rc={"figure.figsize":(15, 4)})
counts_df = pd.DataFrame(data = { 'Characters' : Characters, 'Counts' : [Character_Counts[i] for i in Characters] } )
ax = sns.barplot(x='Characters', y='Counts', data = counts_df)
# [ i for i in Sentences if "ff" in i] 

As is common with the english language, each characters appear with a different frequency. The characters with the highest frequency are:

In [ ]:
counts_df.sort_values(by=['Counts'], ascending = False).head(8)

The characters with the lowest frequency are divided into two categories: 

1. single letters (e.g. q) 
2. double letters (e.g. tt in the word "better")

In the latter case, the signer holds the sign for a given letter while doing "bobbing" or "sliding" motion with their hand to indicate multiple occurances of the letter. The specific letters that are doubled in the data are cc, dd, ee, ff, gg, ll, mm, nn, oo, pp, rr, ss, tt, and zz. 

The single characters with the lowest frequencies are listed below:

In [ ]:
counts_df[ counts_df.Characters.str.len() == 1 ].sort_values(by=['Counts']).head(8)

The double characters with the lowest frequencies are listed below:

In [ ]:
counts_df[ counts_df.Characters.str.len() == 2 ].sort_values(by=['Counts'])

## Data 
<p> Column Labels

In [ ]:
# EDA - Exploratory data analysis
# data.shape # (n,k): n = number of frames in video, k = (k-1 pieces of landmark data) + (1 frame data)

# Landmark data
#### Divide columns into frame number, and landmarks for face, left_hand / right_hand, and pose. 
#    Each of the landmark columns are divided into three based on the x,y,z coordinates. We list
#    total number of xyz columns for landmarks and then the unique landmarks. Links:
#       - Pose:  https://developers.google.com/mediapipe/solutions/vision/pose_landmarker
#       - Hands: https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/index
################################################################################################
C = data.columns
frame = C[0] # 1 frame 
face = [ i for i in C if 'face' in i] # 1404 xyz columns / 468 landmarks
left_hand = [ i for i in C if 'left_hand' in i] # 63 xyz columns / 21 landmarks 
right_hand = [ i for i in C if 'right_hand' in i] # 63 xyz columns / 21 landmarks  
pose = [ i for i in C if 'pose' in i] # 99 xyz columns / 33 landmarks

# quick look
data.head()

# Landmarks
## Import functions 

In [ ]:
# Number of landmarks per object
number_of_landmarks = { 'face' : 468, 'left_hand' : 21, 'right_hand' : 21, 'pose' : 33 }

# simple holder for our landmarks that will mimic the results that we get back from mediapipe but using our data
class Landmarks(object):
    pass

# Returns a "Landmark object" with a list of landmarks for the given object and frame   
def get_landmarks(name, data, frame):
    
    # assert that name is face, left_hand, right_hand, pose
    assert name in {'face', 'left_hand', 'right_hand', 'pose'}
    
    # number of landmarks per object
    n = number_of_landmarks[name]
    
    # loop
    obj = landmark_pb2.NormalizedLandmarkList()
    for i in range(n):
        
        # Column names in the data (dataframe) for the x,y,z coordinates of landmark i
        coordinates = {}
        for j in "xyz":
            column_name = j + '_' + name + '_' + str(i)
            coordinates[j] = data[ column_name ].loc[ data['frame'] == frame ]
        
        # coordinates
        xi,yi,zi = coordinates['x'], coordinates['y'], coordinates['z']
        
        # add to Landmark
        obj.landmark.add( x=xi, y=yi, z=zi )
    
    # return 
    return obj


# Creates all landmarks 
def get_landmarks_data(data, frame):  
    result = Landmarks()
    result.face_landmarks = get_landmarks('face', data, frame)
    result.pose_landmarks = get_landmarks('pose', data, frame)
    result.left_hand_landmarks = get_landmarks('left_hand', data, frame)
    result.right_hand_landmarks = get_landmarks('right_hand', data, frame)
    return result

## Operations

In [ ]:
# Checks if a landmark is not defined
def Landmark_is_nan(X):
    for i in X.landmark:
        if math.isnan(i.x) or math.isnan(i.y) or math.isnan(i.z):
            return True
    return False

# returns <x,y,z> vector for landmark n
def Landmark_vector(X,i):
    return np.array([ X.landmark[i].x,  X.landmark[i].y,  X.landmark[i].z])
    
# Makes a blank landmark of length n
def Landmark_blank(n):
    obj = landmark_pb2.NormalizedLandmarkList()
    for i in range(n):
        obj.landmark.add( x=math.nan, y=math.nan, z=math.nan )
    return obj

# Creates a blank for all landmarks
def blank():  
    result = Landmarks()
    result.face_landmarks = Landmark_blank(468)
    result.pose_landmarks = Landmark_blank(33)
    result.left_hand_landmarks = Landmark_blank(21)
    result.right_hand_landmarks = Landmark_blank(21)
    return result
    
# Makes a landmark undefined
def Landmark_make_nan(X):
    obj = landmark_pb2.NormalizedLandmarkList()
    for i in X.landmark:
        obj.landmark.add( x=math.nan, y=math.nan, z=math.nan )
    return obj


# Shifts a landmark to be centered at the vector w
def shift(X,w):
    obj = landmark_pb2.LandmarkList()
    a = np.array([ X.landmark[0].x , X.landmark[0].y, X.landmark[0].z ]) 
    for i in X.landmark:
        # form vector
        v = np.array([i.x,i.y,i.z])
        # unshift affine shift
        v = v - a + w
        # set coordinates
        obj.landmark.add( x=v[0], y=v[1], z=v[2] )       
    return obj 


# reflects a landmark about the y-axis
def reflect(X):
    obj = landmark_pb2.LandmarkList()
    for i in X.landmark:
        obj.landmark.add( x=(.5-i.x), y=i.y, z=i.z )       
    return obj 

# records whether front or back of hand
def front(X):
    
    # points
    X0 = Landmark_vector(X,0) # base
    X5 = Landmark_vector(X,5) # left 
    X17 = Landmark_vector(X,17) # right
    
    # vectors 
    V1, V2 = X5 - X0, X17 - X0
    
    # set z to 0 
    V1[2], V2[2] = 0,0
    
    # take cross product
    n = np.cross(V1,V2)
    
    return True if n[2] < 0 else False  

## Centering

In [ ]:
# landmarks = get_landmarks_data(video_data(V), 5)
# X = landmarks.right_hand_landmarks
import time 
# x = time.time()
# a = [ center( get_landmarks_data(video_data(V), i).right_hand_landmarks ) for i in V.frames]
# print(time.time() - x)

In [ ]:
# centering using a fast tensorflow implementation
# TODO: 
# - tensors can be made directly from landmark data
def tf_center(X):
    
    # if not defined
    if Landmark_is_nan(X): return X
    
    # Landmarks X0, X5, X17 and corresponding vectors V5 and V17
    X0, X5, X17 = [ tf.convert_to_tensor(Landmark_vector(X,i)) for i in [0,5,17] ]
    V5, V17 = X5 - X0, X17 - X0
    
    # normal vector to the plane of the hand 
    normal = tf.linalg.cross(V5,V17)
    
    # orientation
    orientation = 'front' if 0 < normal[2] else 'back'
    
    # Set image - The matrix B is where you want the vectors normal, V5, and V17 to land.
    A = tf.stack([ normal, V5, V17 ], axis=1)
    if orientation == 'front':
        B = tf.constant([[ 0.00605257,  0.00246374, -0.01169141], 
                         [ 0.04396006, -0.13104451, -0.0048572 ], 
                         [-0.05505851, -0.10182643, -0.04996139]], dtype=tf.float64) # front
    else: 
        B = tf.constant([[ 0.01846211, -0.00295458,  0.02261392], 
                         [ 0.12170109, -0.12008113, -0.11504634], 
                         [ 0.18375406,  0.00450712, -0.14942883]], dtype=tf.float64) # back
    
    # Transformation matrix ( A^-1 * B )
    C = tf.linalg.inv(A) @ B 
    
    # SVD + best orthogonal approximation
    S, U, V = tf.linalg.svd(C)
    O = U @ V
    
    # rescale O
    scale = .01
    mag = tf.norm(normal)
    scale_value = ( scale / mag) ** (1./3) # print(scale_value)
    O = (scale_value) * O
    
    
    obj = landmark_pb2.LandmarkList()
    for i in X.landmark:
            
            # form vector
            p = tf.constant([i.x,i.y,i.z], dtype=tf.float64)
            
            # affine shift
            v = p - X0
            
            # linear transformation
            w = tf.linalg.matvec(O, v)
            
            # set coordinates
            obj.landmark.add( x=w[0], y=w[1], z=w[2] )
            
    return obj
    
    
    



# Recenters a hand based on the outer landmarks 
def center(X):
    
    # if not defined
    if Landmark_is_nan(X): return X

    # Centering
    X0 = Landmark_vector(X,0) # base
    X1 = Landmark_vector(X,1) # left 
    X5 = Landmark_vector(X,5) # center left 
    X9 = Landmark_vector(X,9) # center
    X10 = Landmark_vector(X,10) # center
    X13 = Landmark_vector(X,13) # center right
    X17 = Landmark_vector(X,17) # right
        
    # Basis 1
    V1, V5, V9, V10, V13, V17, = X1-X0, X5-X0, X9-X0, X10-X0, X13-X0, X17-X0
    normal = np.cross(V5, V17)

    # Basis Matrices
    A, B = np.array([normal,V5,V17]),    np.array([[ 0.00605257,  0.00246374, -0.01169141], [ 0.04396006, -0.13104451, -0.0048572 ], [-0.05505851, -0.10182643, -0.04996139]])
    # A, B = np.array([V1,V5,V17]),    np.array([[ 0.06816408, -0.02943546, -0.0151578 ], [ 0.04396006, -0.13104451, -0.0048572 ], [-0.05505851, -0.10182643, -0.04996139]])
    # A1, B1 = np.array([V9,V5,V17]),  np.array([[ 0.12737696, -0.07470322, -0.08873697], [ 0.12005465, -0.11204314, -0.07280551], [ 0.13797943,  0.01287234, -0.11509396]])
    # A1, B1 = np.array([V5,V10,V17]),  np.array([[ 0.12170109, -0.12008113, -0.11504634], [ 0.2935304,  -0.11416346, -0.18584773], [ 0.18375406,  0.00450712, -0.14942883]])
    A1, B1 = np.array([normal,V5,V17]),  np.array([[ 0.01846211, -0.00295458,  0.02261392], [ 0.12170109, -0.12008113, -0.11504634], [ 0.18375406,  0.00450712, -0.14942883]])
    # A, B = np.array([V1,V2,V3]),  np.array([[ 0.04693469, -0.12667215, -0.0208972 ],[ 0.00329348, -0.12194675, -0.03340138],[-0.07567593, -0.0876869 , -0.06737784]])
    
    # print(A1)
    
    # Transformation
    C = (np.linalg.inv(A)).dot(B)
    if not front(X): 
        C = (np.linalg.inv(A1)).dot(B1)
        # print('back')
    
    # SVD
    U, S, V = np.linalg.svd(C, full_matrices=True)
    C = U.dot(V)
    
    # print(np.linalg.det(C))
    
    # resize
    scale = .01
    mag = np.linalg.norm( np.cross(V5,V17), 2)
    rvalue = ( scale / mag) ** (1./3) # print(rvalue)
    D = np.diag(np.full(3,rvalue))
    C = D.dot(C)
    
    # print(np.linalg.det(C), rvalue)
    # print(C)
    
    # print( np.linalg.det(C), (np.linalg.norm(V5) / np.linalg.norm(V17) ), np.dot(V5,V17) / (np.linalg.norm(V5) * np.linalg.norm(V17))  )
    
    obj = landmark_pb2.LandmarkList()
    for i in X.landmark:
            
            # form vector
            v = np.array([i.x,i.y,i.z])
            
            # affine shift
            v = v - X0
            
            # linear transformation
            v = v.dot(C)
            
            # set coordinates
            obj.landmark.add( x=v[0], y=v[1], z=v[2] )
            
    return obj 

# Video class

In [ ]:
# Video Class
class video:           
    def __init__(self,seq_id):
        self.id = seq_id 
        self.label = dataset_labels.loc[ dataset_labels['sequence_id'] == seq_id  ]
        self.phrase = self.label.phrase.iloc[0]
        self.participant = self.label.participant_id.iloc[0]
        self.data = dataset.loc[ seq_id ]
        self.frames = self.data.frame
        self.characters = character_list(self.phrase)
    
    def __repr__(self): 
            a = " Sequence: {0}\n Phrase: {1}\n Signer: {2}\n Frames: {3}\n Dominant Hand: {4}\n Percentage: {5}".format( self.id, 
                                                                                                                          self.phrase,
                                                                                                                          self.participant, 
                                                                                                                          len(self.frames), 
                                                                                                                          dominant_hand(self), 
                                                                                                                          percentage(self))

            return a 
#######################
### Video functions ###
#######################

def sequence(self):
    return self.id

def participant(self):
    return self.participant

def phrase(self):
    return self.phrase

def video_data(self):
    return self.data

def frames(self):
    return self.frames

def characters(self):
    return self.characters

def percentage(self):
    return round( 100 * len( hand_frames(self) ) / len( self.frames ), 2)
    

#######################
#### Dominant Hand ####
#######################

def dominant_hand(self):
    if hasattr(self, 'dominant_hand'):
        return self.dominant_hand
    
    # Select all frames from data
    data = self.data
    frames = data.frame.unique()
    
    # Left / right hand frames
    Lframes = [i for i in frames if not Landmark_is_nan(get_landmarks('left_hand',  data, i))]
    Rframes = [i for i in frames if not Landmark_is_nan(get_landmarks('right_hand', data, i))] 
    
    # Determine dominant hand 
    if len(Lframes) < len(Rframes):
        self.handframes = Rframes
        self.dominant_hand = 'right'
    else:
        self.handframes = Lframes
        self.dominant_hand = 'left'
    
    return self.dominant_hand

### Dominant hand frames
def hand_frames(self):
    dominant_hand(self) # this assigns handframes
    return self.handframes

#######################
### Normalized hand ###
#######################

def normal_hand_data(self):
    
    if hasattr(self, 'normal_hand_data'):
        return self.normal_hand_data, self.hand_position
    
    # hand frames 
    data = self.data
    Hframes = hand_frames(self)
    Dhand = dominant_hand(self)
    
    # loop 
    L, P = [], [] 
    for frame in Hframes:

        # Select landmarks for dominant hand
        X = get_landmarks_data(data, frame).right_hand_landmarks if Dhand == 'right' else get_landmarks_data(data, frame).left_hand_landmarks

        # First store position
        P.append( list(Landmark_vector(X,0)) )

        # center landmark
        X = center(X)

        # List
        L.append( [ list(Landmark_vector(X,i)) for i in range(1,21) ] )

    # assign to video
    self.normal_hand_data, self.hand_position = L, P
    
    # return 
    return self.normal_hand_data, self.hand_position
          

In [ ]:
V = video(seq_id)
D = video_data(V)

In [ ]:
print(V)

## Exploritory Data Analysis

We now do some basic exploritory data analysis on the 1000 sequences in the specific data file labeled '33432165'. For each sequence, we record:
1. number of frames 
2. percent of frames with dominant hand 

In [ ]:
# Todo

# Labeled Data - Method 1 
## Clustering: k-means time series

In [ ]:
GoodR = [4, 7, 10, 14, 15, 34, 82, 90, 100, 127, 131, 193, 219, 223, 228, 238, 339, 376, 386, 401, 457, 504, 510, 525, 528, 535, 538, 548, 570, 599, 611, 638, 651, 658, 692, 711, 784, 792, 804, 822, 835, 854, 864, 924, 940]
V = video(VideoID[924])
print(V.phrase, k, len(V.characters)) 
# k = find_k(V)
# Clustering(V,k)
# TensorFlow_Label(V,M)
Label(V,M)

In [ ]:
%pprint

### Finding value of k

In [ ]:
# function returns WSS score for k values from 1 to kmax
def calculate_WSS(self):
    
    # data
    points, _ = normal_hand_data(self)
    points = np.array(points).reshape(len(points), -1) 
    kmax = len(self.frames) - 1

    sse = []
    for k in range(1, kmax+1):
        kmeans = KMeans(n_clusters = k, n_init = 10).fit(points)
        sse.append(kmeans.inertia_)
    return sse


def calculate_sil(self):
    
    # data
    points, _ = normal_hand_data(self)
    points = np.array(points).reshape(len(points), -1) 
    kmax = len(self.frames) - 1

    # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
    sil = []
    for k in range(2, kmax+1):
        kmeans = KMeans(n_clusters = k, n_init = 10).fit(points)
        labels = kmeans.labels_
        sil.append(silhouette_score(points, labels, metric = 'euclidean'))
    
    return sil 

#### k - Weighted square error 

In [ ]:
# function for k
def find_k(V):
    # weighted square error for k = 0 to len(V.frames) 
    kmetric = calculate_WSS(V)

    # plot
    # plt.plot(kmetricz)

    # Elbow method 
    kmetricz = [ i * len(kmetric) / kmetric[0] for i in kmetric ] # rescale x and y axis to be the same increments
    slopes = [ kmetricz[i+1] - kmetricz[i] for i in range(len(kmetricz)-1) ] # look at slopes
    slop = [pair for pair in enumerate(slopes)] 
    slop.sort( key = lambda x : abs(x[1]+1) ) # find slope that is closest to -1

    # value of k
    k = slop[0][0]
    
    return k

#### k - silowette method

In [ ]:
# silowette score for k = 0 to len(V.frames) 
silmetric = calculate_sil(V)

# plot 
# plt.plot(silmetric)

# value of k (max silowette score)
a = max(silmetric[6:])
k0 = silmetric.index(a)
print(k0)

## Cluster Functions 

In [ ]:
#### TODO
# - GridsearchCV function below
# - Find method for picking out strongest **re-occuring** clusters in kmeans data and labeling them


# Label the data using a k-means time series clustering algorithm
def label_clusters(self, k, method):
    
    # data
    data, _ = normal_hand_data(self)
    data = np.array(data).reshape(len(data), -1) 
    
    # Select Model
    if method   == 'kmeans': model = KMeans(n_clusters = k, n_init = 10, max_iter = 20)
    elif method == 'ts':     model = TimeSeriesKMeans(n_clusters=k, n_init = 10, max_iter=20)
    elif method == 'dtw':    model = TimeSeriesKMeans(n_clusters=k, n_init = 10, metric="dtw", max_iter=20)
    model.fit(data)
   
    # return
    return model.labels_



# label refining
def refine_clusters(C):
    
    # tolerance threshold
    d = 2
    
    # add buffer
    C = list(C) + ['*']
    
    # Relabel data with distinct increasing labels
    t, k, prev = 0, 0, -1
    L = []
    for i in C:
        if i == prev: 
            k += 1
        else:
            if k < d: 
                L += ['?'] * k 
            else:
                L += [t] * k
                t += 1
            k = 1
        prev = i
    
    # labels
    return L



# Assigning clusters
def Clustering(self, k):
    
    # parameters 
    n = 10
    method = 'kmeans'
    
    # run k-means labeling n time 
    clusters = defaultdict(list)
    for j in range(n):
        labs = label_clusters(self, k, method)
        rlabs = refine_clusters(labs)
        for i,x in enumerate(rlabs):
            clusters[i] += [x]
    
    # average the labels assigned to each frame
    average_clusters, CLabels = [], []
    for i in clusters:
        x = max(clusters[i], key=clusters[i].count) # max label assigned to entry i
        c = clusters[i].count(x) # count of max label assigned to entry i (used for confidence)
        average_clusters += [x]
        CLabels += [c]
    
    # refine label (make sure that they are ordered)
    rclusters = refine_clusters(average_clusters)
    
    # assign
    self.clusters = rclusters
    
    # return 
    return self.clusters 

# Model 

### Load Data 

In [ ]:
# load Data
with open("LabledData40.json", 'r') as f:
    train = json.load(f)

train_x, train_y = [], []
for seq_id in train:
    
    # load date
    V = video(int(seq_id))
    N, p = normal_hand_data(V)
    pN = train[seq_id]
    
    # loop
    for i in range(len(N)):
        if pN[i] != '?':
            train_x += [ N[i] ]
            train_y += [ Encode[ pN[i] ] ]

In [ ]:
XTrain, Xtest, YTrain, YTest = train_test_split(train_x, train_y, test_size=0.3) 

### Tensorflow model 

In [ ]:
# Model
k = len( Characters )
M = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# fit model
M.fit( XTrain, YTrain, epochs = 500, verbose=0 )

### Accuracy

In [ ]:
Loss, Acctrain = M.evaluate(XTrain, YTrain, verbose=2)
Loss, Acctest = M.evaluate(Xtest, YTest, verbose=2)
print('\nTrain accuracy:', Acctrain)
print('\nTest accuracy:', Acctest)

In [ ]:
X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])

# prediction (encoded as an integer)
YPred = [ np.argmax( X.predict( np.array([i]) , verbose=0) ) for i in Xtest ]

In [ ]:
bad = pd.DataFrame()
bad['y_true'] = [Decode[i] for i in YTest]
bad['y_pred'] = [Decode[i] for i in YPred]
bad['incorrect'] = (bad['y_pred'] == bad['y_true'])

# counts
A = bad.groupby(bad.columns.tolist(),as_index=False).size()

A.sort_values('size',ascending=False).head(50)

# Predictions 

In [ ]:
def precomputed_labels(self):
    
    # check if assigned 
    if not hasattr(self, 'precomputedlabels'):
        
        # precomp
        with open("LabledData5.json", 'r') as f:
            LabledDict = json.load(f)
        
        # assert
        seq_id = str(V.id)
        if seq_id not in LabledDict: print('Labels are not precomputed')
        
        # assign
        self.precomputedlabels = LabledDict[seq_id]
            
    return self.precomputedlabels
    
       
### Predictions
#  Methods = 'tensor_flow', 'precomputed', or 'kmeans'
#
def predict(self, frame):
    
    method = 'precomputed'
    
    # TensorFlow 
    if method == 'tensor_flow': 
        
        # data
        data, _ = normal_hand_data(V)
        data = np.array([ data[frame] ])
    
        # individual prediction
        X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])
        p = X.predict( data, verbose=0)
    
        # prediction (encoded as an integer)
        pred = np.argmax(p)
    
        # answer
        ans = Decode[pred]
    
    # Kmeans
    elif method == 'kmeans':
        
        # shift 
        d = 0 
        
        # kmeans prediction
        x = self.data_labels[frame]
        
        # answer
        ans = '?' if (x == '?' or x < d) else self.characters[x-d]
    
    # Precomputed 
    elif method == 'precomputed':
        
        # precomputed labels
        precomp = V.data_labels 
        
        # answer
        ans = precomp[frame]
                                          
    # return
    return ans 

# Labeling Clusters

In [ ]:
# TensorFlow Labels
# Takes as input a video V = self and model M with labeled clusters
def TensorFlow_Label(self, M):
    
    # initialize 
    clumps = self.clusters
    characters = self.characters
    frames, _ = normal_hand_data(self)
    
    # Need softmax later for predictions
    X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])
    
    ### Average predictions by cluster
    Predictions = {} 
    cluster_labels = list({i for i in clumps if i != '?'})
    for c in cluster_labels: 
        
        # indices + frames
        c_indices = [ i for i,x in enumerate(clumps) if x == c ] 
        c_frames  = [ frames[i] for i in c_indices ] 
        
        # Average prediction
        Predictions[c] = sum( X.predict( np.array([f]), verbose=0) for f in c_frames )[0] / len(c_frames)
    
        
    ### Dynamic programing
    # initialize
    n, m = len(characters), len(cluster_labels)
    
    ## score function 
    # Given a letter (e.g. 'a') and cluster label c (e.g. 0) returns a score of the letter for that clump 
    def score( letter, c ):
        x = Encode[ letter ]
        s = Predictions[c][x]
        return s
    
    # dp array
    dp = defaultdict( lambda : (0,[]) ) 
    for i, letter in enumerate(characters):
        best, labs = 0, []
        for j in range(m):
            
            # update best previous score
            if best < dp[(i-1,j-1)][0]:
                best, labs = dp[(i-1,j-1)]
            
            # score at ij
            c = cluster_labels[j]
            score_ij = best + score( letter, c )
            labs_ij = labs + [ c ] 
            
            # update dp
            dp[(i,j)] = ( score_ij, labs_ij ) 
    
    
    # optimal score
    m, labs = max([ dp[(n-1,j)] for j in range(m) ])

    # updating labels 
    Y_Labels = list(V.clusters)
    for i, x in enumerate(Y_Labels):
        if x in labs:
            Y_Labels[i] = characters[ labs.index(x) ]
        else:
            Y_Labels[i] = '?'
    # assign
    self.data_labels = Y_Labels
    
    # return clusters
    return self.data_labels   





# TensorFlow Labels
# Takes as input a video V = self and model M with labeled clusters
def Label(self, M):
    
    # initialize 
    characters = self.characters
    frames, _ = normal_hand_data(self)
    n, m = len(characters), len(frames)
    
    # Need softmax later for predictions
    X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])
    
    # Predictions
    Predictions = [] 
    for x in frames: 
        Predictions += [ X.predict(np.array([x]), verbose=0)[0] ]
    
    ## score function 
    # Given a letter (e.g. 'a') and cluster label c (e.g. 0) returns a score of the letter for that clump 
    def score( letter, i ):
        # print(letter,i)
        x = Encode[ letter ]
        s = Predictions[i][x]
        return s
    
    
    ### Prefix sums
    D = {}
    for c in characters:
        
        # initialize
        D[c] = defaultdict(int)
        pre, L = 0, []
        
        # loop
        for j in range(m):
            
            # update prefix sum
            pre += score(c,j)
            
            # loop over past prefix sum and compute score of i,j
            for i,x in enumerate(L):
                D[c][(i,j)] = (pre - x) / math.sqrt(j-i)
            
            # update list of prefixes
            L += [pre]
        
        # normalize scores
        maxim = max(D[c][y] for y in D[c])
        
        # normalize all scores
        for x in D[c]:
            D[c][x] /= maxim
            
    
    ### Dynamic programing
    
    # dp array
    dp = defaultdict( lambda : (0,[]) ) 
    for i, letter in enumerate(characters):
        
        # best + labels
        best, labs = 0, []
        
        # loop
        for j in range(m):
            
            # update best previous score
            if best < dp[(i-1,j-1)][0]:
                # print(best,labs)
                best, labs = dp[(i-1,j-1)]
            
            # loop over k
            for k in range(j+1,m):
                
                # length of interval
                l = k-j
                
                # average letter score for the frames j to k
                score_jk = best + D[letter][(j,k)]
                #print( (pre[letter][k] - pre[letter][j]) / l , letter, j,k)
                
                if dp[(i,k)][0] < score_jk:
                    
                    # update labels
                    labs_jk = labs + [ (j,k,letter) ]
                    
                    # update dp
                    dp[(i,k)] = ( score_jk, labs_jk ) 
    
    
    # optimal score
    maxim, labs = max([ dp[(n-1,j)] for j in range(m) ])

    # updating labels 
    newlabels = ['?'] * m
    for i, j, letter in labs:
        for k in range(i,j+1):
            newlabels[k] = letter
                
    # assign
    self.data_labels = newlabels
    
    # return clusters
    return newlabels 




# Plotting 

In [ ]:
def draw_landmarks(landmarks,image,show_pose=True,show_face_contour=True,show_face_tesselation=True,show_left_hand=True,show_right_hand=True):
    annotated_image = image.copy()
    results = landmarks
    if show_face_tesselation:
        mp_drawing.draw_landmarks(
            annotated_image,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
    if show_face_contour:
        mp_drawing.draw_landmarks(
            annotated_image,
            results.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
    if show_pose:
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.
            get_default_pose_landmarks_style())
    if show_left_hand:
        mp_drawing.draw_landmarks(
            annotated_image,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles
            .get_default_hand_landmarks_style())
    if show_right_hand:
        mp_drawing.draw_landmarks(
            annotated_image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles
            .get_default_hand_landmarks_style())
    return annotated_image


## Plot: single frame

In [ ]:
annotated_image = np.zeros((1024,1024,3),dtype=np.uint8)
Data, frame = video_data(V), 1
landmarks = get_landmarks_data(Data,frame)
#show_image(draw_landmarks(landmarks,annotated_image))

## Plot: Scrolling through frames 

In [ ]:
# Scrolling through frames
def show(V):
    # initialize the data from Video V
    Data = video_data(V)
    Frames = frames(V)
    # retrun frame function
    def show_frame(frame):
        landmarks = get_landmarks_data(Data,frame)
        show_image(draw_landmarks(landmarks,annotated_image), figsize=(7,7), title=f'frame: {frame} [{frame+1} of {len(Frames)}]')
    return show_frame

#f = show(V)
#interact(f, frame = widgets.IntSlider( min=0, max=len(V.frames)-1, step=0, value=0, layout=widgets.Layout(width='1000px')))

## Plot: Dominant hand (only frame that it is visible)

In [ ]:
# Scrolling through image frames
# prediction_method = 'tflow' or 'kmeans'
def show_hands(V):    
    
    # initialize
    Hframes = hand_frames(V)
    frames = V.frames
    phrase = V.phrase
    Data = video_data(V)
       
    # show function
    def show_frame(frame):
        # Only do frames with dominant hand
        frame = Hframes[frame]

        # Get landmark data
        landmarks = get_landmarks_data(Data, frame)

        # Remove face and pose landmarks 
        landmarks.face_landmarks = Landmark_make_nan(landmarks.face_landmarks)
        landmarks.pose_landmarks = Landmark_make_nan(landmarks.pose_landmarks)

        # set dominant hand (if left handed we reflect about the y-axis)
        if dominant_hand == 'left': landmarks.right_hand_landmarks = reflect(landmarks.left_hand_landmarks)

        # Plot original landmark and centered hand 
        landmarks.left_hand_landmarks = shift(landmarks.right_hand_landmarks, np.array([.2,.7,0]))
        landmarks.right_hand_landmarks = shift( center( landmarks.right_hand_landmarks ), np.array([.6,.7,0] ) )
        show_image(draw_landmarks(landmarks,annotated_image),figsize=(6,6),title=f'frame [{frame} of {len(frames)-1}] {phrase:50} Prediction: { predict(V,frame) }')
        #show_image(draw_landmarks(landmarks,annotated_image),figsize=(6,6),title=f'frame [{frame+1} of {len(frames)}] {phrase:50} phrase: { V.phrase }')
        # print(f'showing frame: {frames[frame]}')
    
    return show_frame

f = show_hands(V)
interact(f, frame=widgets.IntSlider(min=0, max=len(V.frames)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))

In [ ]:
# change label
def changelabel(self, i, letter):
    self.data_labels[i] = letter
    
for i in range(149,150):
    changelabel(V,i,'?')

### Pickle

In [ ]:
# pickle function
def pickle(D,V):
    seq_id = str(V.id)
    Frames = V.frames
    D[seq_id] = [ predict(V,i) for i in Frames ]

In [ ]:
# WRITE
# with open("LabledData40.json", 'w') as f:
    # json.dump(pickles, f, indent=2) 

# LOAD
# with open("LabledData40.json", 'r') as f:
    # pickles = json.load(f)

In [ ]:
pickle(pickles,V)

In [ ]:
len(pickles)

# Dictionary Stats

In [ ]:
Counts = defaultdict(int)
for p in pickles:
    Vp = video(int(p))
    print(Vp.phrase)
    for i in Vp.characters:
        Counts[i] += 1

sorted([i for i in Counts])

In [ ]:
# notes 804 is misspelled
GoodDictionary = [ i for i,x in enumerate(VideoID) if str(x) in pickles ]
BadDictionary = [i for i in GoodR if i not in GoodDictionary ]
BadDictionary

# Sentence Reconstruction

In [ ]:
from Levenshtein import distance as lev

# tolerence + video
def reconstruct_sentence(V,M):
    
    # initialize k 
    k = find_k(V)

    # add softmax layer for predictions
    data, _ = normal_hand_data(V)
    X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])
    
    # clustering 
    clusters = Clustering(V,k)
    cluster_labels = sorted(list({i for i in clusters if i != '?'}))
    
    # predictions
    ans = ''
    prev = ''
    for c in cluster_labels:
        
        index_c = [ i for i,x in enumerate(clusters) if x == c] 
        
        # individual prediction
        p = sum( X.predict( np.array([ data[i] ]), verbose=0) for i in index_c )
    
        # prediction (encoded as an integer)
        pred = np.argmax(p)
        char = Decode[pred]
        
        # add to ans (if not same as previous letter)
        if char != prev: 
            ans += Decode[pred]
        prev = char
    
    # match words with phrase using Levenshtein distance
    words = V.phrase.split()
    
    # loop - do dp instead
    # w = ''
    # rans = ''
    # for word in words:
        # for i in ans:
            # n = w + i
            # if lev(w, word) < lev(n, word):
                #rans += w + ' '
                #w = i
                #break 
    #rans += w
    
    # return 
    return ans, V.phrase


# TensorFlow Labels
# Takes as input a video V = self and model M with labeled clusters
def Corg(self, M):
    
    # initialize 
    frames, _ = normal_hand_data(self)
    n, m = len(Characters), len(frames)
    
    # Need softmax later for predictions
    X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])
    
    # Predictions
    Predictions = [ X.predict(np.array([x]), verbose=0)[0] for x in frames ]
    
    ## score function 
    # Given a letter (e.g. 'a') and cluster label c (e.g. 0) returns a score of the letter for that clump 
    def score( letter, i ):
        # print(letter,i)
        x = Encode[ letter ]
        s = Predictions[i][x]
        return s
    

    ### Prefix sums
    D = {}
    for c in Characters:
        
        # initialize
        D[c] = defaultdict(int)
        pre, L = 0, []
        
        # loop
        for j in range(m):
            
            # update prefix sum
            pre += score(c,j)
            
            # loop over past prefix sum and compute score of i,j
            for i,x in enumerate(L):
                D[c][(i,j)] = (pre - x) 
            
            # update list of prefixes
            L += [pre]
        
        # normalize scores
        # maxim = max(D[c][y] for y in D[c])
        
        # normalize all scores
        # for x in D[c]:
            # D[c][x] /= maxim
            
    
    ### Dynamic programming
    # dp array
    dp = defaultdict( lambda : (0,[]) )
    for i in range(m):
        
        # update past
        if dp[i][0] < dp[i-1][0]:
            dp[i] = dp[i-1]
        
        # update future
        for j in range(i+2, m):
            
            # loop
            for c in Characters:
                c_score = D[c][(i+1,j)]
                if dp[j][0] < c_score + dp[i][0]:
                    dp[j] = ( c_score + dp[i][0], dp[i][1] + [(i,j,c)] )
    
    # optimal score
    score, labs = dp[m-1]
    
    # new labels
    newlabels = ['?'] * m
    for i, j, letter in labs:
        for k in range(i,j+1):
            newlabels[k] = letter 
    
    # self.data_labels = newlabels
            
    # sentence
    s = ''.join(i[2] for i in labs)
    
    #return s, labs, D, [ Decode[np.argmax(i)] for i in Predictions]
    return s




def Basic_Sentence(self, M): 
    
    # initialize 
    frames, _ = normal_hand_data(self)
    
    # Need softmax later for predictions
    X = tf.keras.Sequential([ M, tf.keras.layers.Softmax() ])
    
    # letter labels
    Predictions = [ X.predict(np.array([x]), verbose=0)[0] for x in frames ]
    L = [ Decode[np.argmax(i)] for i in Predictions]
    
    # tolerance threshold
    d = 2
    
    # add buffer
    L = list(L) + ['*']
    
    # Relabel data with distinct increasing labels
    c, prev = 0, '*'
    s = ''
    for i in L:
        if i == prev: 
            c += 1
        else:
            if d <= c: 
                s += prev
            c = 1
        prev = i
    
    # labels
    return s

        

In [ ]:
a,labs,D, preds = Corg(V,M)
a, labs, preds

In [ ]:
D['e'][(135,140)], D['b'][(122,129)]

In [ ]:
a = Basic_Sentence(V,M)

In [ ]:
# x = reconstruct_sentence(V,M)

In [ ]:
correct = ''.join(V.phrase.split())
x = 'cocmunicatsthrounghemail'

N = len(correct)
D = lev(correct, a)
print((N-D)/N)

In [ ]:
def metric(self,M):
    
    s1 = ''.join(self.phrase.split())
    s2 = Corg(self, M)
    
    N, D = len(s1), lev(s1, s2)
    
    return (N-D) / N

In [ ]:
avg = 0
for i,x in enumerate(BadDictionary):
    V = video(VideoID[x])
    t = metric(V,M)
    avg += t
    print( V.phrase, Basic_Sentence(V,M), t, avg / (i+1))

In [ ]:


"""
# Scrolling through image frames
def show_frame(frame):
    frame = Hframes[frame]
    landmarks = get_landmarks_data(frame)
    landmarks.face_landmarks = Landmark_make_nan(landmarks.face_landmarks)
    landmarks.pose_landmarks = Landmark_make_nan(landmarks.pose_landmarks)
    if dominant_hand == 'left':
        landmarks.right_hand_landmarks = reflect(landmarks.left_hand_landmarks)
        landmarks.left_hand_landmarks = Landmark_make_nan(landmarks.left_hand_landmarks)
    landmarks.right_hand_landmarks = shift( center( landmarks.right_hand_landmarks, frame), np.array([.5,.7,0]))
    # show_image(draw_landmarks(landmarks,annotated_image),figsize=(9,9),title=f'Tflow Prediction: {predict(frame):10} Actual: {prediction(frame):10} frame: {frames[frame]} [{frame+1} of {len(frames)}]')
    show_image(draw_landmarks(landmarks,annotated_image),figsize=(9,9),title=f'Tflow Prediction: {predict(frame):10} {phrase:30} frame: {frames[frame]} [{frame+1} of {len(frames)}]')
    #print(f'showing frame: {frames[frame]}')


interact(show_frame, frame=widgets.IntSlider(min=0, max=len(Hframes)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))
"""


# Scripts for computing data 

# Tensorflow labeled alphabet

In [ ]:
# path to alphabet landmark data
path = '/Volumes/My Passport for Mac/ASL-labled-data/archive/asl_alphabet_train/Landmarkdata.json'

# open data as pickles
with open(path, 'r') as f:
    train_alphabet = json.load(f)

In [ ]:
def CreateLandmark(c):
    
    # create a blank landmark
    X = blank()
    
    # create new landmark for left hand 
    obj = landmark_pb2.LandmarkList()
    
    # fill in new landmark
    for v in c: obj.landmark.add( x=v[0], y=v[1], z=v[2] ) # add other landmarks
    
    # set left hand to be obj
    X.left_hand_landmarks = obj
    X.left_hand_landmarks = center(X.left_hand_landmarks)
    # X.left_hand_landmarks = shift( X.left_hand_landmarks, np.array([.5,.7,0]))
    
    return X 

In [ ]:
annotated_image = np.zeros((1024,1024,3),dtype=np.uint8)
landmarks = CreateLandmark(train_alphabet['A'][0])
landmarks.left_hand_landmarks = shift( landmarks.left_hand_landmarks, np.array([.5,.7,0]))
show_image(draw_landmarks(landmarks,annotated_image))

In [ ]:
# Scrolling through frames
letter = 'N'
Frames = [ x for x in train_alphabet[letter] if x ]
def show(frame):
    # return frame function
    landmarks = CreateLandmark(Frames[frame])
    landmarks.left_hand_landmarks = shift( landmarks.left_hand_landmarks, np.array([.5,.7,0]))
    show_image(draw_landmarks(landmarks,annotated_image), figsize=(7,7), title=f'frame: {frame}')


interact(show, frame = widgets.IntSlider( min=0, max=len(Frames)-1, step=0, value=0, layout=widgets.Layout(width='1000px')))

# storing nice values for x,y,z

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
X_train, y_train = [], []
for letter in alphabet:
    Frames = [ x for x in train_alphabet[letter] if x ]
    for x in Frames:
        landmarks = CreateLandmark(x)
        L = [ [i.x, i.y, i.z ] for i in landmarks.left_hand_landmarks.landmark ]
        X_train += [ L[1:] ]
        y_train += [ letter ]
  

In [ ]:
y_train = [ Encode[i.lower()] for i in y_train ]

In [ ]:
# Model
k = len( Characters )
M1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M1.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# fit model
M1.fit( XTrain , YTrain, epochs = 20, verbose=0 )

In [ ]:
XTrain, Xtest, YTrain, YTest = train_test_split(X_train, y_train, test_size=0.3) 

In [ ]:
Loss, Acctrain = M1.evaluate(XTrain, YTrain, verbose=2)
Loss, Acctest = M1.evaluate(Xtest, YTest, verbose=2)
print('\nTrain accuracy:', Acctrain)
print('\nTest accuracy:', Acctest)

In [ ]:
# train = [ train_x, train_y ]
# with open("LabeledAlphabetData.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    # json.dump(train, f, indent=2)

In [ ]:
'''
file = '33432165'
frame = 47
s = Video_ID[4] 
annotated_image = np.zeros((1024,1024,3),dtype=np.uint8)
landmarks = get_landmarks_data(frame)
show_image(draw_landmarks(landmarks,annotated_image))
X = fix_z(landmarks.right_hand_landmarks)

# Centering
X0 = Landmark_vector(X,0) # base
X1 = Landmark_vector(X,1) # left 
X5 = Landmark_vector(X,5) # center left 
X9 = Landmark_vector(X,9) # center
X13 = Landmark_vector(X,13) # center right
X17 = Landmark_vector(X,17) # right

# correct z
# X2[2] = predict_z( X2, Landmark_vector(X,10), Landmark_vector(X,11), Landmark_vector(X,12) )
# X2[2] = X.landmark[10].z  
    
# Basis 1
V1, V5, V9, V13, V17 = X1-X0, X5-X0, X9-X0, X13-X0, X17-X0

print( V1, V5, V9, V13, V17 ) 
'''

# Collecting complete videos

In [ ]:
# Select video data
'''
GoodL, GoodR = [],[]
for j,x in enumerate(Video_ID):
    
    data = dataset.loc[x]
    data_labels = dataset_labels.loc[dataset_labels['sequence_id'] == x ]
    
    # All frames
    frames = data.frame.unique()

    # Right / Left hand frames
    Lframes = [i for i in frames if not Landmark_is_nan(get_landmarks('left_hand',i))]
    Rframes = [i for i in frames if not Landmark_is_nan(get_landmarks('right_hand',i))]
    
    # Dominant hand 
    if len(Lframes) < len(Rframes):
        Hframes = Rframes
        dominant_hand = "right"
    else:
        Hframes = Lframes
        dominant_hand = "left"
    
    print("Signer in %sth sequence is %s-handed and we have %s hand frames which comprise %s%% of the frames" % (j, dominant_hand, len(Hframes), round(100 * len(Hframes) / len(frames), 2)))
    if len(Hframes) == len(frames):
          if dominant_hand == 'right': GoodR += [j]
          else: GoodL += [j]
''' 

In [ ]:
Vids = [ video(VideoID[i]) for i in GoodR ]

In [ ]:
#[print(i) for i in Vids]
Goodchars = set(sum([[j[0] for j in i.characters] for i in Vids], []))

In [ ]:
GoodFrames = [ normal_hand_data(i) for i in Vids]

In [ ]:
F = sum([i[0] for i in GoodFrames], [])

In [ ]:
k = len(Goodchars)
model = KMeans(n_clusters = k, n_init = 10, max_iter = 100)
data = np.array(F).reshape(len(F), -1) 
model.fit(data)

In [ ]:
centroids = [i.reshape(20,3) for i in model.cluster_centers_]

In [ ]:
centroids[0]

In [ ]:
def landmark_from_hand_data(c):
    
    # create a blank landmark
    X = blank()
    
    # create new landmark for left hand 
    obj = landmark_pb2.LandmarkList()
    
    # fill in new landmark
    obj.landmark.add( x=0, y=0, z=0 ) # origin is wrist
    for v in c: obj.landmark.add( x=v[0], y=v[1], z=v[2] ) # add other landmarks
    
    # set left hand to be obj
    X.left_hand_landmarks = obj
    X.left_hand_landmarks = shift(X.left_hand_landmarks, np.array([.5,.7,0]))
    
    return X   

In [ ]:
annotated_image = np.zeros((1024,1024,3),dtype=np.uint8)
landmarks = landmark_from_hand_data(centroids[14])
# show_image(draw_landmarks(landmarks,annotated_image))

# Extras

## Old Labeling method

In [ ]:
'''
def label_data(self):
    
    # check
    # if hasattr(self, 'data_labels'): return self.data_labels
           
    # list of characters
    chars = characters(self)
    
    # data
    data, _ = normal_hand_data(self)
    data = np.array(data).reshape(len(data), -1) 
    
    # Phase 1: Set up time series k-means algorithm for distinct symbols
    k = len(set(i[0] for i in chars))
    # model = TimeSeriesKMeans(n_clusters=k, metric="dtw", max_iter=20)
    # model.fit(data)
    # Alternate Phase 1
    model = KMeans(n_clusters = k, n_init = 10, max_iter = 20)
    model.fit(data)
    print(model.labels_)
    
    # one hot labels and run again
    data_labels = np.array([ i for i in model.labels_ ]).reshape(-1,1)
    
    # Phase 2: second round of clustering based on the sentence 
    k = len(chars)
    Model = TimeSeriesKMeans(n_clusters = k, metric = "dtw", max_iter = 20)
    Model.fit(data_labels)
    
    # return as np.array
    self.data_labels = Model.labels_
   
    # return
    return self.data_labels
'''   

'''
# refines the clustering data to make more uniform predictions
def refine_labels(C, d = 4, k = 1):
    
    # Relabel data with distinct increasing labels
    t, prev = 0, C[0]
    L = []
    for i in C:
        if i != prev:
            t += 1
            prev = i
        L += [t]
    
    # Count the occurences of each label
    Counts = Counter(L)
    # print(len(Counts))
    
    # loop and replace occurences with less than 4 with -1
    temp_labels = []
    for i in L:
        x = i if (d <= Counts[i]) else -1   
        temp_labels += [ x ]
    
    Counts = Counter(temp_labels)
    print(len(Counts))
    
    # Look for clumps in the -1s
    #A, L = [], []
    #for i,x in enumerate(temp_labels):
        #if x == -1: L += [i]
        #elif L: 
            #A += [L]
            #L = []
            
    # Longest sets of -1   
    A.sort(key=len)
    
    # Match m and n
    #n = len(V.characters) + 1
    #m = len(set(temp_labels))
    #while A and m < n:
        #t += 1
        #clump = A.pop()
        #for i in clump:
            #temp_labels[i] = t
        #m += 1
    
    # Replace labels with -1 with '?' and reindex labels 
    t, v = -1, {}
    Labels = []
    for i in temp_labels:
        if i == -1:
            Labels += [ '?' ]
        else:
            if i not in v: 
                v[i] = 0
                t += 1
            Labels += [t]
    
    # assign + return
    return Labels
'''

In [ ]:
'''
#########################################
#### Measurements using hand model ######
#########################################

import sympy 
frame = 128
annotated_image = np.zeros((1024,1024,3),dtype=np.uint8)
landmarks = get_landmarks_data(frame)

# Triangle for hand 
X = landmarks.right_hand_landmarks

# z-prediction
from sympy import *
from sympy import sqrt
z1, z2 = symbols('z1, z2')

Measurements = {
 (0, 1) : 0.19917,
 (1, 2) : 0.18034,
 (2, 3) : 0.12843,
 (3, 4) : 0.09624,
 (0, 5) : 0.41711,
 (5, 6) : 0.16196,
 (6, 7) : 0.10277,
 (7, 8) : 0.08568,
 (0, 9) : 0.40355,
 (9, 10) : 0.18095,
 (10, 11) : 0.11845,
 (11, 12) : 0.09565,
 (0, 13) : 0.3855,
 (13, 14) : 0.16929,
 (14, 15) : 0.10963,
 (15, 16) : 0.09177,
 (0, 17) : 0.37061,
 (17, 18) : 0.13211,
 (18, 19) : 0.08178,
 (19, 20) : 0.07294 }

# z-prediction - given points 
def triangle_prediction(X,Y):
    
    # dummy variable for z value of X,Y
    V = [ X[0], X[1], z1 ] 
    W = [ Y[0], Y[1], z2 ]
    
    # Create Equations
    E1 = Eq( np.dot(V,W), sqrt(np.dot(V,V) * np.dot(W,W)) * (0.6781246776238563) ) # dot product a * b = |a||b| cos( theta )
    E2 = Eq( np.dot(V,V), np.dot(W,W) * (1.1254837879588218)**2 ) # Ratio
    
    # set determinant = 0 and solve 
    ans = list(nonlinsolve([E1,E2], [z1, z2]))
    # print(ans)
    
    # new z-coordinates
    z1_new, z2_new = float(ans[0][0]), float(ans[0][1])
    
    # return
    return z1_new, z2_new 


def fix_z(X):
    
    # shift to origin
    X = shift( X, np.array([0, 0, 0]) )
    
    ### Fundamental triangle 0--5--17
    # Build new (0,5) and (0,17) vectors
    X5, X17 = Landmark_vector(X, 5), Landmark_vector(X, 17) 
    X5[2], X17[2] = triangle_prediction(X5,X17)
    # print(X5[2], X17[2])
      
    # initialize measurements
    base = np.linalg.norm(X5)
    measurement_base = Measurements[(0,5)]
    
    # initizalize obj
    obj = landmark_pb2.LandmarkList()
    obj.landmark.add( x=0, y=0, z=0 ) # set 0 landmark to origin
    
    # loop 
    for i,j in Measurements:
        if (i,j) == (0,5): 
            obj.landmark.add( x=X5[0], y=X5[1], z=X5[2])
        elif (i,j) == (0,17):
            obj.landmark.add( x=X17[0], y=X17[1], z=X17[2] )
        else:
            # vector connecting i and j (use variable for z)
            Xi, Xj = Landmark_vector(obj, i), Landmark_vector(X, j)
            V = Xj - Xi
            V = [V[0], V[1], z1]
            
            # Equation
            E = Eq( sqrt(np.dot(V,V)) / base, Measurements[(i,j)] / measurement_base )

            # solution 
            corr = list(solveset(E, z1))
            # print( j, corr, Xj[2], Xi[2] + corr[0])
            
            corr = corr[0]
                  
            # convert to float if real, else set to 0
            corr = float(corr) if corr.is_real else 0

            # set coordinates
            obj.landmark.add( x=Xj[0], y=Xj[1], z=Xi[2] + corr )
    
    return obj
        

# Recenters a hand based on the outer landmarks 
def centered(X, frame):
    
    # if not defined
    if Landmark_is_nan(X): return X

    # Centering
    X0 = Landmark_vector(X,0) # base
    X1 = Landmark_vector(X,1) # left 
    X5 = Landmark_vector(X,5) # center left 
    X9 = Landmark_vector(X,9) # center
    X13 = Landmark_vector(X,13) # center right
    X17 = Landmark_vector(X,17) # right 
        
    # Possible basis vectors 
    Base = [ (1, X1-X0), (5, X5-X0), (9, X9-X0), (13, X13-X0), (17, X17-X0) ] 
    
    # Sort basis vectors by z-coordinates
    Base = sorted( Base, key=lambda x: x[1][2] )
    
    # Select vectors 
    L = [ Base[0] ] + [ sorted(Base[1:4], key=lambda x: abs(x[1][2] - Base[0][1][2]) * abs(x[1][2] - Base[4][1][2]))[2] ] + [ Base[4] ] 
    
    print(L)
    
    D = { 
    1  : [ 0.06816408, -0.02943546,  0.        ], 
    5  : [ 0.04396006, -0.13104451, -0.01766506], 
    9  : [ 0.00693706, -0.12849939, -0.04018739], 
    13 : [-0.02584845, -0.11831534, -0.04380703],
    17 : [-0.05505851, -0.10182643, -0.04391743] }
    
    # Basis Matrices
    A, B = np.array([i[1] for i in L]),  np.array([ D[i[0]] for i in L ])
    A1, B1 = np.array([ i[1] for i in Base[0:3]]),  np.array([ D[i] for i in [1,5,9]])
    print(A1,B1)
    
    # Transformation
    C = (np.linalg.inv(A)).dot(B)
    C1 = (np.linalg.inv(A1)).dot(B1)
    
    print(np.linalg.det(C), np.linalg.det(C1))
    # print( np.linalg.det(C), (np.linalg.norm(V5) / np.linalg.norm(V17) ), np.dot(V5,V17) / (np.linalg.norm(V5) * np.linalg.norm(V17))  )
    
    obj = landmark_pb2.LandmarkList()
    for i in X.landmark:
            
            # form vector
            v = np.array([i.x,i.y,i.z])
            
            # affine shift
            v = v - X0
            
            # linear transformation
            v = v.dot(C)
            
            # set coordinates
            obj.landmark.add( x=v[0], y=v[1], z=v[2] )
            
    return obj       

#########################################
###### Entropy (letter from phrase) #####
#########################################

sphrase = phrase.replace(" ", "")

def letter_predict(frame):
    i = math.floor( len(phrase) * frame / len(frames) ) 
    return phrase[i]

def letter_prediction(frame):
    i = bisect.bisect_left(T,frame)
    if len(sphrase) <= i: i = len(sphrase)-1
    return sphrase[i]

# fingers
hand_obj = { 'thumb' : [1,2,3,4], 'pointer' : [6,7,8], 'middle' : [10,11,12], 'ring' : [14,15,16], 'pinky' : [18,19,20], 'palm' : [0,5,9,13,17] }

# measure the sum of the distances between handlandmarks in any two frames 
def entropy(frame):
    
    L = []
    for frame in frames:
    
        # select hand landmarks from frame
        if dominant_hand == 'right': X = center( get_landmarks_data(frame).right_hand_landmarks, frame )
        else: X = center( get_landmarks_data(frame).left_hand_landmarks, frame )
    
        # check if defined
        if Landmark_is_nan(X): continue
        
        Eb, Ef = {}, {}
        for obj in hand_obj:
        
            # loop over landmarks and add to entropy
            db, df = 0, 0
            for i in hand_obj[obj]:
            
                # a = coordinates of i^th landmark in frame,  b = coordinate of i^th landmark in (frame - 1)
                a = Landmark_vector(X,i) 
                b = Landmark_vector(Y,i)
                c = Landmark_vector(Z,i) 
            
                # remove z-values
                a[2], b[2], c[2] = 0,0,0
            
                # distance 
                db += np.linalg.norm(a-b,2)
                df += np.linalg.norm(b-c,2)
            
            # add to entropy
            Eb[obj] = db
            Ef[obj] = df
    
    # total entropy
    teb = round( sum(Eb[i] for i in Eb) * 100 , 3 )
    tef = round( sum(Ef[i] for i in Ef) * 100 , 3 )
    
    # total entropy
    total_entropy = min(teb,tef)
    
    # print("heyo", teb, tef)
   
    # return entropy
    return total_entropy
'''        
        

In [ ]:
'''
# List all characters from sentence
# characters = V.characters
# frames = V.frames

# Good frames
# GoodFrames = [i for i in frames if prediction(V,i) != '?']

# training data
# train_data = [ V.normal_hand_data + V.hand_position for i in GoodFrames]
# train_predictions = [ Encode[prediction(V,i)] for i in GoodFrames ]
train_x, train_y = pickles
train_y = [ Encode[i] for i in train_y ]

# Model
k = len( Characters )
M = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# fit model
M.fit( train_x , train_y, epochs = 100, verbose=0 )
'''

In [ ]:
Kaggle dataset
# https://www.kaggle.com/datasets/grassknoted/asl-alphabet